In [257]:
import torch
import torchvision
import torchvision.transforms as transforms

In [258]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 4 * 4, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
net = Net()
net.load_state_dict(torch.load("LeNET_300_100_MNIST_Model"))

In [259]:
from torchsummary import summary
device=torch.device("cpu")
model=Net().to(device)
summary(model, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 300]          77,100
            Linear-6                  [-1, 100]          30,100
            Linear-7                   [-1, 10]           1,010
Total params: 110,782
Trainable params: 110,782
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.42
Estimated Total Size (MB): 0.47
----------------------------------------------------------------


In [260]:
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())
    print(param_tensor, "\t", net.state_dict()[param_tensor])

conv1.weight 	 torch.Size([6, 1, 5, 5])
conv1.weight 	 tensor([[[[-0.0192,  0.2511,  0.2183,  0.0210,  0.0190],
          [-0.0522,  0.3645,  0.3561,  0.5513,  0.3440],
          [ 0.0598,  0.5070,  0.6116,  0.6220,  0.2953],
          [ 0.2985,  0.1301,  0.4567,  0.1638,  0.2149],
          [-0.0246,  0.3206,  0.1249,  0.1288, -0.1632]]],


        [[[-0.1775, -0.0977,  0.0400,  0.2623,  0.2931],
          [ 0.0318, -0.1988, -0.1956,  0.1583,  0.1678],
          [-0.2439, -0.2983, -0.0792,  0.2235,  0.3461],
          [-0.0280, -0.2769, -0.1121,  0.0926,  0.3560],
          [-0.1970, -0.1645,  0.1938,  0.2115,  0.1824]]],


        [[[ 0.1362,  0.2118, -0.2154, -0.2076, -0.2334],
          [ 0.2467, -0.0821, -0.1670, -0.0738,  0.2066],
          [ 0.2564,  0.1952,  0.2048,  0.3864,  0.0963],
          [ 0.0786,  0.3546,  0.3418, -0.0168,  0.1037],
          [ 0.0178, -0.2039, -0.1759, -0.0146, -0.0048]]],


        [[[ 0.0299,  0.1694, -0.0029, -0.1827, -0.0718],
          [-0.1909, -

In [261]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)

In [262]:
i=0
for parameter in net.parameters():
    i+=1
    print(i," ",parameter.shape,"\n ",parameter,"\n\n")

1   torch.Size([6, 1, 5, 5]) 
  Parameter containing:
tensor([[[[-0.0192,  0.2511,  0.2183,  0.0210,  0.0190],
          [-0.0522,  0.3645,  0.3561,  0.5513,  0.3440],
          [ 0.0598,  0.5070,  0.6116,  0.6220,  0.2953],
          [ 0.2985,  0.1301,  0.4567,  0.1638,  0.2149],
          [-0.0246,  0.3206,  0.1249,  0.1288, -0.1632]]],


        [[[-0.1775, -0.0977,  0.0400,  0.2623,  0.2931],
          [ 0.0318, -0.1988, -0.1956,  0.1583,  0.1678],
          [-0.2439, -0.2983, -0.0792,  0.2235,  0.3461],
          [-0.0280, -0.2769, -0.1121,  0.0926,  0.3560],
          [-0.1970, -0.1645,  0.1938,  0.2115,  0.1824]]],


        [[[ 0.1362,  0.2118, -0.2154, -0.2076, -0.2334],
          [ 0.2467, -0.0821, -0.1670, -0.0738,  0.2066],
          [ 0.2564,  0.1952,  0.2048,  0.3864,  0.0963],
          [ 0.0786,  0.3546,  0.3418, -0.0168,  0.1037],
          [ 0.0178, -0.2039, -0.1759, -0.0146, -0.0048]]],


        [[[ 0.0299,  0.1694, -0.0029, -0.1827, -0.0718],
          [-0.1909, -0

In [263]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %f %%' % (
    100 * correct / total))

old_accuracy=correct/total

Accuracy of the network on the test images: 98.110000 %


In [264]:
(net.state_dict()["fc1.weight"]).shape

torch.Size([300, 256])

In [265]:
(net.state_dict()["fc2.weight"]).shape

torch.Size([100, 300])

In [266]:
beta=1
prob=[0]*len(net.state_dict()["fc1.weight"])
norm=[0]*len(net.state_dict()["fc1.weight"])
dino=[0]*len(net.state_dict()["fc2.weight"])
for p in range(len(net.state_dict()["fc1.weight"])):
    
    ##########################################################################################
    #######################   Here give p=1 for norm1, p=2 for norm2 #########################
    ##############    use norm[p]=torch.sum(net.state_dict()["fc1.weight"][p]) for sum #######
    
    norm[p]=torch.norm(net.state_dict()["fc1.weight"][p], p=1)
for i in range(len(net.state_dict()["fc2.weight"])):
    for q in range(len(net.state_dict()["fc1.weight"])):
        dino[i]+=(net.state_dict()["fc2.weight"][i][q].item())*F.relu(beta*norm[q])
    
for p in range(len(net.state_dict()["fc1.weight"])):
    for i in range(len(net.state_dict()["fc2.weight"])):
        ele=(net.state_dict()["fc2.weight"][i][p].item())*F.relu(beta*norm[p])/(dino[i])
        if(ele>prob[p]):
            prob[p]=ele.item()

print("prob size=",len(prob))

prob size= 300


In [267]:
(net.state_dict()["fc2.weight"]).shape

torch.Size([100, 300])

In [268]:
u=torch.zeros([len(net.state_dict()["fc2.weight"]), len(net.state_dict()["fc2.weight"][0])], dtype=torch.float)
print(u.shape)

torch.Size([100, 300])


In [269]:
from random import choices

###************************************************************************************************************###
###************************************************************************************************************###
###************************************************************************************************************###

#### Here we are choosing the compression ration ####

compr= 0.05
m=int(compr*len(net.state_dict()["fc1.weight"]))
C=choices([i for i in range(len(prob))], prob, k=m)
print(C)

[184, 294, 299, 179, 5, 125, 45, 78, 8, 77, 251, 64, 119, 140, 192]


In [270]:
for q in C:
    for i in range(len(net.state_dict()["fc2.weight"])):
        u[i][q]+=net.state_dict()["fc2.weight"][i][q]/(m*prob[q])

In [271]:
for i in range(len(u)):
    for j in range(len(u[0])):
        net.state_dict()["fc2.weight"][i][j]=u[i][j]

In [272]:
print(net.state_dict()["fc2.weight"].shape)

torch.Size([100, 300])


In [273]:
check=[0]*len(net.state_dict()["fc1.weight"])
for i in C:
    check[i]=1
print(check)

[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]


In [274]:
temp=torch.stack([net.state_dict()["fc1.weight"][i][:] for i in range(len(check)) if(check[i]==1)], dim=0)
net.state_dict()["fc1.weight"].resize_((len(temp), len(temp[0])))
for i in range(len(temp)):
    for j in range(len(temp[0])):
        net.state_dict()["fc1.weight"][i][j]=temp[i][j]

In [275]:
print(len(temp)," ",len(temp[0]))

15   256


In [276]:
print(temp.shape, " ", net.state_dict()["fc1.weight"].shape)

torch.Size([15, 256])   torch.Size([15, 256])


In [277]:
# for i in range(len(temp)):
#     for j in range(len(temp[0])):
#         print(temp[i][j]," ",net.state_dict()["fc1.weight"][i][j],end="\n")

In [278]:
temp=torch.stack([net.state_dict()["fc1.bias"][i] for i in range(len(check)) if(check[i]==1)])
print(temp.shape)

torch.Size([15])


In [279]:
net.state_dict()["fc1.bias"].resize_(len(temp))
print(net.state_dict()["fc1.bias"].shape)

torch.Size([15])


In [280]:
for i in range(len(temp)):
    net.state_dict()["fc1.bias"][i]=temp[i]
    
for i in range(len(temp)):
    print(net.state_dict()["fc1.bias"][i]," ",temp[i])

tensor(-0.0318)   tensor(-0.0318)
tensor(0.0534)   tensor(0.0534)
tensor(0.0529)   tensor(0.0529)
tensor(0.0466)   tensor(0.0466)
tensor(0.0373)   tensor(0.0373)
tensor(-0.0519)   tensor(-0.0519)
tensor(0.0434)   tensor(0.0434)
tensor(0.0384)   tensor(0.0384)
tensor(-0.0445)   tensor(-0.0445)
tensor(-0.0050)   tensor(-0.0050)
tensor(-0.0539)   tensor(-0.0539)
tensor(0.0111)   tensor(0.0111)
tensor(0.0607)   tensor(0.0607)
tensor(-0.0119)   tensor(-0.0119)
tensor(0.0507)   tensor(0.0507)


In [281]:
net.state_dict()["fc2.weight"].shape

torch.Size([100, 300])

In [282]:
#Modify fc2.weights
rows=[]
for i in range(len(net.state_dict()["fc2.weight"])):
    row=torch.stack([net.state_dict()["fc2.weight"][i][j] for j in range(len(check)) if(check[j]==1)])
    rows.append(row)

In [283]:
print(len(rows)," ",rows[0].shape)

100   torch.Size([15])


In [284]:
temp=torch.stack([rows[i] for i in range(len(rows))], dim=0)
print(temp.shape)

torch.Size([100, 15])


In [285]:
net.state_dict()["fc2.weight"].resize_((len(temp), len(temp[0])))
for i in range(len(temp)):
    for j in range(len(temp[0])):
        net.state_dict()["fc2.weight"][i][j]=temp[i][j]

In [286]:
net.state_dict()["fc2.weight"].shape

torch.Size([100, 15])

In [287]:
net.state_dict()["fc2.bias"].shape

torch.Size([100])

## Now prune Layer 3, (100 nodes)

In [288]:
beta=1
prob=[0]*len(net.state_dict()["fc2.weight"])
norm=[0]*len(net.state_dict()["fc2.weight"])
dino=[0]*len(net.state_dict()["fc3.weight"])
for p in range(len(net.state_dict()["fc2.weight"])):
    
    ##########################################################################################
    #######################   Here give p=1 for norm1, p=2 for norm2 #########################
    ##############    use norm[p]=torch.sum(net.state_dict()["fc1.weight"][p]) for sum #######
    
    norm[p]=torch.norm(net.state_dict()["fc2.weight"][p], p=1)
    
for i in range(len(net.state_dict()["fc3.weight"])):
    for q in range(len(net.state_dict()["fc2.weight"])):
        dino[i]+=(net.state_dict()["fc3.weight"][i][q].item())*F.relu(beta*norm[q])
    
for p in range(len(net.state_dict()["fc2.weight"])):
    for i in range(len(net.state_dict()["fc3.weight"])):
        ele=(net.state_dict()["fc3.weight"][i][p].item())*F.relu(beta*norm[p])/(dino[i])
        if(ele>prob[p]):
            prob[p]=ele.item()

print("prob size=",len(prob))

prob size= 100


In [289]:
(net.state_dict()["fc3.weight"]).shape

torch.Size([10, 100])

In [290]:
u=torch.zeros([len(net.state_dict()["fc3.weight"]), len(net.state_dict()["fc3.weight"][0])], dtype=torch.float)
print(u.shape)

torch.Size([10, 100])


In [291]:
from random import choices

###************************************************************************************************************###
###************************************************************************************************************###
###************************************************************************************************************###

#### Here we are choosing the compression ration ####

compr= 0.05

m=int(compr*len(net.state_dict()["fc2.weight"]))
C=choices([i for i in range(len(prob))], prob, k=m)
print(C)

[72, 21, 69, 69, 22]


In [292]:
for q in C:
    for i in range(len(net.state_dict()["fc3.weight"])):
        u[i][q]+=net.state_dict()["fc3.weight"][i][q]/(m*prob[q])

In [293]:
for i in range(len(u)):
    for j in range(len(u[0])):
        net.state_dict()["fc3.weight"][i][j]=u[i][j]

In [294]:
print(net.state_dict()["fc3.weight"].shape)

torch.Size([10, 100])


In [295]:
check=[0]*len(net.state_dict()["fc2.weight"])
for i in C:
    check[i]=1
print(check)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [296]:
temp=torch.stack([net.state_dict()["fc2.weight"][i][:] for i in range(len(check)) if(check[i]==1)], dim=0)
net.state_dict()["fc2.weight"].resize_((len(temp), len(temp[0])))
for i in range(len(temp)):
    for j in range(len(temp[0])):
        net.state_dict()["fc2.weight"][i][j]=temp[i][j]

In [297]:
print(temp.shape, " ", net.state_dict()["fc2.weight"].shape)

torch.Size([4, 15])   torch.Size([4, 15])


In [298]:
temp=torch.stack([net.state_dict()["fc2.bias"][i] for i in range(len(check)) if(check[i]==1)])
print(temp.shape)

torch.Size([4])


In [299]:
net.state_dict()["fc2.bias"].resize_(len(temp))
print(net.state_dict()["fc2.bias"].shape)

torch.Size([4])


In [300]:
for i in range(len(temp)):
    net.state_dict()["fc2.bias"][i]=temp[i]
    
for i in range(len(temp)):
    print(net.state_dict()["fc2.bias"][i]," ",temp[i])

tensor(0.0081)   tensor(0.0081)
tensor(-0.0266)   tensor(-0.0266)
tensor(0.0128)   tensor(0.0128)
tensor(-0.0255)   tensor(-0.0255)


In [301]:
net.state_dict()["fc3.weight"].shape

torch.Size([10, 100])

In [302]:
#Modify fc3.weights
rows=[]
for i in range(len(net.state_dict()["fc3.weight"])):
    row=torch.stack([net.state_dict()["fc3.weight"][i][j] for j in range(len(check)) if(check[j]==1)])
    rows.append(row)

In [303]:
print(len(rows)," ",rows[0].shape)

10   torch.Size([4])


In [304]:
temp=torch.stack([rows[i] for i in range(len(rows))], dim=0)
print(temp.shape)

torch.Size([10, 4])


In [305]:
net.state_dict()["fc3.weight"].resize_((len(temp), len(temp[0])))
for i in range(len(temp)):
    for j in range(len(temp[0])):
        net.state_dict()["fc3.weight"][i][j]=temp[i][j]

In [306]:
net.state_dict()["fc3.weight"].shape

torch.Size([10, 4])

In [307]:
net.state_dict()["fc3.bias"].shape

torch.Size([10])

In [308]:
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())
    print(param_tensor, "\t", net.state_dict()[param_tensor])

conv1.weight 	 torch.Size([6, 1, 5, 5])
conv1.weight 	 tensor([[[[-0.0192,  0.2511,  0.2183,  0.0210,  0.0190],
          [-0.0522,  0.3645,  0.3561,  0.5513,  0.3440],
          [ 0.0598,  0.5070,  0.6116,  0.6220,  0.2953],
          [ 0.2985,  0.1301,  0.4567,  0.1638,  0.2149],
          [-0.0246,  0.3206,  0.1249,  0.1288, -0.1632]]],


        [[[-0.1775, -0.0977,  0.0400,  0.2623,  0.2931],
          [ 0.0318, -0.1988, -0.1956,  0.1583,  0.1678],
          [-0.2439, -0.2983, -0.0792,  0.2235,  0.3461],
          [-0.0280, -0.2769, -0.1121,  0.0926,  0.3560],
          [-0.1970, -0.1645,  0.1938,  0.2115,  0.1824]]],


        [[[ 0.1362,  0.2118, -0.2154, -0.2076, -0.2334],
          [ 0.2467, -0.0821, -0.1670, -0.0738,  0.2066],
          [ 0.2564,  0.1952,  0.2048,  0.3864,  0.0963],
          [ 0.0786,  0.3546,  0.3418, -0.0168,  0.1037],
          [ 0.0178, -0.2039, -0.1759, -0.0146, -0.0048]]],


        [[[ 0.0299,  0.1694, -0.0029, -0.1827, -0.0718],
          [-0.1909, -

In [309]:
i=0
for parameter in net.parameters():
    i+=1
    print(i," ",parameter.shape,"\n ",parameter,"\n\n")

1   torch.Size([6, 1, 5, 5]) 
  Parameter containing:
tensor([[[[-0.0192,  0.2511,  0.2183,  0.0210,  0.0190],
          [-0.0522,  0.3645,  0.3561,  0.5513,  0.3440],
          [ 0.0598,  0.5070,  0.6116,  0.6220,  0.2953],
          [ 0.2985,  0.1301,  0.4567,  0.1638,  0.2149],
          [-0.0246,  0.3206,  0.1249,  0.1288, -0.1632]]],


        [[[-0.1775, -0.0977,  0.0400,  0.2623,  0.2931],
          [ 0.0318, -0.1988, -0.1956,  0.1583,  0.1678],
          [-0.2439, -0.2983, -0.0792,  0.2235,  0.3461],
          [-0.0280, -0.2769, -0.1121,  0.0926,  0.3560],
          [-0.1970, -0.1645,  0.1938,  0.2115,  0.1824]]],


        [[[ 0.1362,  0.2118, -0.2154, -0.2076, -0.2334],
          [ 0.2467, -0.0821, -0.1670, -0.0738,  0.2066],
          [ 0.2564,  0.1952,  0.2048,  0.3864,  0.0963],
          [ 0.0786,  0.3546,  0.3418, -0.0168,  0.1037],
          [ 0.0178, -0.2039, -0.1759, -0.0146, -0.0048]]],


        [[[ 0.0299,  0.1694, -0.0029, -0.1827, -0.0718],
          [-0.1909, -0

  Parameter containing:
tensor([[ 0.0038,  0.0061,  0.0112,  0.0201],
        [ 0.0069,  0.0075, -0.0122, -0.0076],
        [-0.0024,  0.0084,  0.0240, -0.0002],
        [-0.0007, -0.0050, -0.0106, -0.0198],
        [ 0.0094,  0.0073,  0.0186,  0.0142],
        [ 0.0007, -0.0067,  0.0138,  0.0044],
        [ 0.0025, -0.0012, -0.0004, -0.0108],
        [-0.0022,  0.0005, -0.0144,  0.0172],
        [-0.0050, -0.0012, -0.0362, -0.0207],
        [ 0.0039, -0.0030,  0.0107,  0.0170]], requires_grad=True) 


10   torch.Size([10]) 
  Parameter containing:
tensor([-0.0838,  0.0204, -0.0172,  0.0138,  0.0053, -0.0699,  0.0305,  0.0152,
        -0.0514, -0.0417], requires_grad=True) 




In [310]:
i=0
for parameter in net.parameters():
    i+=1
    if(i<5):
        parameter.requires_grad=False

In [311]:
i=0
for parameter in net.parameters():
    i+=1
    print(i," ",parameter.shape,"\n ",parameter,"\n\n")

1   torch.Size([6, 1, 5, 5]) 
  Parameter containing:
tensor([[[[-0.0192,  0.2511,  0.2183,  0.0210,  0.0190],
          [-0.0522,  0.3645,  0.3561,  0.5513,  0.3440],
          [ 0.0598,  0.5070,  0.6116,  0.6220,  0.2953],
          [ 0.2985,  0.1301,  0.4567,  0.1638,  0.2149],
          [-0.0246,  0.3206,  0.1249,  0.1288, -0.1632]]],


        [[[-0.1775, -0.0977,  0.0400,  0.2623,  0.2931],
          [ 0.0318, -0.1988, -0.1956,  0.1583,  0.1678],
          [-0.2439, -0.2983, -0.0792,  0.2235,  0.3461],
          [-0.0280, -0.2769, -0.1121,  0.0926,  0.3560],
          [-0.1970, -0.1645,  0.1938,  0.2115,  0.1824]]],


        [[[ 0.1362,  0.2118, -0.2154, -0.2076, -0.2334],
          [ 0.2467, -0.0821, -0.1670, -0.0738,  0.2066],
          [ 0.2564,  0.1952,  0.2048,  0.3864,  0.0963],
          [ 0.0786,  0.3546,  0.3418, -0.0168,  0.1037],
          [ 0.0178, -0.2039, -0.1759, -0.0146, -0.0048]]],


        [[[ 0.0299,  0.1694, -0.0029, -0.1827, -0.0718],
          [-0.1909, -0

In [312]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

In [313]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [314]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print("[{}, {}] loss: {}".format
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Reraining')

[1, 20] loss: 0.023021982908248903
[1, 40] loss: 0.023016324281692504
[1, 60] loss: 0.02300759208202362
[1, 80] loss: 0.023002485036849974
[1, 100] loss: 0.02300203216075897
[1, 120] loss: 0.022990919947624207
[1, 140] loss: 0.022991984486579894
[1, 160] loss: 0.022975107192993165
[1, 180] loss: 0.022915960907936097
[1, 200] loss: 0.022831026315689087
[1, 220] loss: 0.022692428112030028
[1, 240] loss: 0.02241683328151703
[1, 260] loss: 0.02210299623012543
[1, 280] loss: 0.021761979937553407
[1, 300] loss: 0.021335302710533143
[2, 20] loss: 0.020960787177085875
[2, 40] loss: 0.020614586353302003
[2, 60] loss: 0.020065431356430054
[2, 80] loss: 0.019469966769218444
[2, 100] loss: 0.018671338558197022
[2, 120] loss: 0.017407630145549774
[2, 140] loss: 0.016655201256275176
[2, 160] loss: 0.016318272948265076
[2, 180] loss: 0.016311046421527864
[2, 200] loss: 0.01567469835281372
[2, 220] loss: 0.015681805014610292
[2, 240] loss: 0.015585947632789612
[2, 260] loss: 0.015396187722682952
[2, 2

[16, 40] loss: 0.0028666595220565795
[16, 60] loss: 0.0030114098489284515
[16, 80] loss: 0.00277943155169487
[16, 100] loss: 0.002877113975584507
[16, 120] loss: 0.0025847939625382424
[16, 140] loss: 0.0028934344947338106
[16, 160] loss: 0.0028148945793509484
[16, 180] loss: 0.0026612533777952193
[16, 200] loss: 0.0027105706557631493
[16, 220] loss: 0.0028820991665124895
[16, 240] loss: 0.0027652222514152527
[16, 260] loss: 0.00273809564858675
[16, 280] loss: 0.0026928236335515975
[16, 300] loss: 0.002642790123820305
[17, 20] loss: 0.0028995209112763405
[17, 40] loss: 0.0025532514825463296
[17, 60] loss: 0.002678573802113533
[17, 80] loss: 0.0025704989433288574
[17, 100] loss: 0.0029427814930677414
[17, 120] loss: 0.002704580843448639
[17, 140] loss: 0.0025761005878448488
[17, 160] loss: 0.0025381143540143965
[17, 180] loss: 0.002682006761431694
[17, 200] loss: 0.0025592432096600533
[17, 220] loss: 0.0026362598836421965
[17, 240] loss: 0.002658356934785843
[17, 260] loss: 0.00273212037

In [315]:
torch.save(net.state_dict(), "LeNET_x_y_MNIST_Model_Norm_1_prunded_Fine_Tuned")

In [316]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 4 * 4, len(net.state_dict()["fc1.weight"]))
        self.fc2 = nn.Linear(len(net.state_dict()["fc1.weight"]), len(net.state_dict()["fc2.weight"]))
        self.fc3 = nn.Linear(len(net.state_dict()["fc2.weight"]), 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
net = Net()
net.load_state_dict(torch.load("LeNET_x_y_MNIST_Model_Norm_1_prunded_Fine_Tuned"))

In [317]:
from torchsummary import summary
device=torch.device("cpu")
model=Net().to(device)
summary(model, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                   [-1, 15]           3,855
            Linear-6                    [-1, 4]              64
            Linear-7                   [-1, 10]              50
Total params: 6,541
Trainable params: 6,541
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.02
Estimated Total Size (MB): 0.07
----------------------------------------------------------------


In [318]:
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())
    print(param_tensor, "\t", net.state_dict()[param_tensor])

conv1.weight 	 torch.Size([6, 1, 5, 5])
conv1.weight 	 tensor([[[[-0.0192,  0.2511,  0.2183,  0.0210,  0.0190],
          [-0.0522,  0.3645,  0.3561,  0.5513,  0.3440],
          [ 0.0598,  0.5070,  0.6116,  0.6220,  0.2953],
          [ 0.2985,  0.1301,  0.4567,  0.1638,  0.2149],
          [-0.0246,  0.3206,  0.1249,  0.1288, -0.1632]]],


        [[[-0.1775, -0.0977,  0.0400,  0.2623,  0.2931],
          [ 0.0318, -0.1988, -0.1956,  0.1583,  0.1678],
          [-0.2439, -0.2983, -0.0792,  0.2235,  0.3461],
          [-0.0280, -0.2769, -0.1121,  0.0926,  0.3560],
          [-0.1970, -0.1645,  0.1938,  0.2115,  0.1824]]],


        [[[ 0.1362,  0.2118, -0.2154, -0.2076, -0.2334],
          [ 0.2467, -0.0821, -0.1670, -0.0738,  0.2066],
          [ 0.2564,  0.1952,  0.2048,  0.3864,  0.0963],
          [ 0.0786,  0.3546,  0.3418, -0.0168,  0.1037],
          [ 0.0178, -0.2039, -0.1759, -0.0146, -0.0048]]],


        [[[ 0.0299,  0.1694, -0.0029, -0.1827, -0.0718],
          [-0.1909, -

In [319]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %f %%' % (
    100 * correct / total))

new_accuracy=correct/total

Accuracy of the network on the test images: 93.690000 %


In [320]:
accuracy_drop = old_accuracy - new_accuracy
print("accuracy drop = ", accuracy_drop, " and it's ",accuracy_drop/old_accuracy*100,"% drop")

accuracy drop =  0.04420000000000002  and it's  4.5051472836611985 % drop
